In [1]:
# import os
# from huggingface_hub import InferenceClient
# from datetime import datetime

# client = InferenceClient(
#     api_key=os.environ["HF_TOKEN"],
# )

# # System prompt for pseudo code generation
# SYSTEM_PROMPT = """You are an expert pseudo code generator. Your task is to:
# 1. Analyze control narrative documents
# 2. Extract key processes and workflows
# 3. Convert them into clear, well-structured pseudo code
# 4. Use proper indentation and logical flow
# 5. No actual code syntax - only pseudo code notation
# 6. No explanations or additional text - only the pseudo code
# 7. And no extras, just pseudo code based on the given information."""

# user_input = "Write pseudo code based on this control narrative document:"

# # Convert bytes to string
# full_text_str = "\n".join([text.decode("utf-8") if isinstance(text, bytes) else text for text in full_text])

# # print("\n" + "="*80)
# # print("PSEUDO CODE GENERATION - HuggingFace API")
# # print("="*80)
# # print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
# # print(f"Model: openai/gpt-oss-20b")
# # print("="*80 + "\n")

# completion = client.chat.completions.create(
#     model="openai/gpt-oss-20b",
#     messages=[
#         {
#             "role": "user",
#             "content": SYSTEM_PROMPT + "\n\n" + user_input + "\n\n" + full_text_str
#         }
#     ],
#     max_tokens=2048,
#     temperature=0.7,
# )

# response_text = completion.choices[0].message.content

# print(response_text)


In [2]:
import pymupdf

doc = pymupdf.open("document.pdf")

In [3]:
text_blocks = []
image_blocks = []

In [4]:
def extract_pdf_content(pdf_path):
    doc = pymupdf.open(pdf_path)

    text_collection = []
    image_collection = []

    for page_num, page in enumerate(doc):
        page_dict = page.get_text("dict")

        for block_id, block in enumerate(page_dict["blocks"]):
            if block["type"] == 0:
                text = " ".join(
                    span["text"]
                    for line in block["lines"]
                    for span in line["spans"]
                ).strip()

                if text:
                    text_collection.append({
                        "id": f"text_p{page_num}_b{block_id}",
                        "content": text,
                        "metadata": {
                            "page": page_num,
                            "bbox": block["bbox"],
                            "modality": "text"
                        }
                    })
        
        for img_id, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base = doc.extract_image(xref)

            image_collection.append({
                "id": f"image_p{page_num}_{img_id}",
                "image_bytes": base["image"],
                "format": base["ext"],
                "metadata": {
                    "page": page_num,
                    "xref": xref,
                    "modality": "image"
                }
            })
    return text_collection, image_collection

In [5]:
texts, images = extract_pdf_content("document.pdf")

print(len(texts), "text chunks")
print(len(images), "images")


845 text chunks
46 images


In [6]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="devstral-2512",
    temperature=0.2,
)

response = llm.invoke("You are just coding bot, so dont give any other content just asked code. write code for simple neural network in pytorch")
print(response.content)


```python
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Example usage
input_size = 10
hidden_size = 5
output_size = 1
model = SimpleNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop (example)
for epoch in range(100):
    # Generate random data
    inputs = torch.randn(32, input_size)
    targets = torch.randn(32, output_size)

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.

In [1]:
from langchain_mistralai import ChatMistralAI
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

2025-12-26 23:00:47.807208: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-26 23:00:47.807497: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-26 23:00:47.851041: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-26 23:00:49.487321: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
loader = PyPDFLoader("document.pdf")
docs = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200
)
chunks = text_splitter.split_documents(docs)

In [4]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


/tmp/ipykernel_33367/661755571.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [16]:
llm = ChatMistralAI(model="devstral-2512", temperature=0.0)

In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are a STRICT PSEUDOCODE GENERATOR.

You MUST FOLLOW THESE RULES EXACTLY:

1. OUTPUT ONLY REAL PROCESS / LOGIC / CONTROL STEPS.
2. NEVER convert titles, headers, document names, revision info, dates, or metadata
   into pseudocode. IGNORE:
   - Title page
   - Revision history
   - Version numbers
   - Dates
   - Table of contents
   - Section headings
   - Descriptive paragraphs
   - Narratives
3. DO NOT OUTPUT STRINGS. DO NOT print document text. DO NOT use quotes.
4. Extract ONLY actionable logic:
   - controls
   - decision conditions
   - sequences
   - state handling
   - alarms
   - inputs / outputs
   - transitions
5. Represent behavior, processes, state transitions and operations ONLY.
6. ALL CONTROL KEYWORDS MUST BE CAPITAL:
   BEGIN, END, IF, ELSE, ELSE IF, FOR, WHILE, RETURN, FUNCTION
7. Use meaningful variable names that match process names in the document.
8. Use indented, clean, algorithmic pseudocode.
9. No explanation, no text, no narration. Only pseudocode.
10. If the document only contains descriptive text, extract logic implied within it.
11. If something is not logical or executable, ignore it.

<context>
{context}
</context>

TASK: {input}
""")


In [18]:
from langchain_core.runnables import RunnableParallel
rag_chain = (RunnableParallel( context=retriever, input=RunnablePassthrough() ) | prompt | llm )

In [19]:
response = rag_chain.invoke("Generate pseudocode based on the document content")
print(response.content)

Based on the provided document content, here's the extracted pseudocode for the Process Automation System (PAS) control logic:

```
FUNCTION InitializeChromatographySkid(skidId)
    BEGIN
        SET skidState TO "Initializing"
        LOAD recipeParameters FOR skidId
        ASSIGN feedSUSV FROM recipeFormula
        ASSIGN collectionSUSV FROM recipeFormula
        SET skidState TO "Idle"
    END

FUNCTION StartChromatographyPhase(skidId)
    BEGIN
        IF skidState == "Idle" THEN
            SET skidState TO "Startup"
            SEND startCommand TO skidId
            MONITOR batchStatus
            SET skidState TO "Feed"
        ELSE
            RAISE "Skid not in idle state"
        END IF
    END

FUNCTION MonitorBatchStatus(skidId)
    BEGIN
        WHILE skidState != "Shutdown" AND skidState != "Abort" DO
            IF operatorAcknowledgementReceived THEN
                SET skidState TO "Shutdown"
            ELSE IF upstreamSkidState == "Idle" THEN
                SET sk